In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
from show_images import mostrar_imagen, mostrar_outfit
from outfit_cleaning import replace_codes,contiene_numeros,tiene_mas_de_dos,cumple_condiciones
from generate_outfits import generate_new_outfits_optimized,recomendar_outfits
import re
import networkx as nx
import ast
import pickle


In [2]:
directorio_actual = os.getcwd()
directorio_actual

'c:\\Users\\joelg\\defdatathon\\mango_datathon_2023\\src'

## Data Loading

In [3]:
# Obtén el directorio actual (src)
directorio_actual = os.getcwd()

# Sube un nivel para llegar al directorio principal
directorio_principal = os.path.dirname(directorio_actual)

# Define la ruta relativa al archivo outfit_data.csv
ruta_outfit_data = os.path.join(directorio_principal,'datathon', 'dataset', 'outfit_data.csv')
ruta_product_data = os.path.join(directorio_principal,'datathon', 'dataset', 'product_data.csv')

# Lee el archivo usando la ruta relativa
outfit_data = pd.read_csv(ruta_outfit_data)
product_data = pd.read_csv(ruta_product_data)

## Data Cleaning

In [4]:
product_data['des_product_aggregated_family'] = product_data.apply(
    lambda row: 'Footwear' if row['des_product_family'] == 'Footwear' else row['des_product_aggregated_family'],
    axis=1)

# Cambiar valores en 'des_line' de 'VIOLETA' a 'SHE'
product_data['des_line'] = product_data['des_line'].replace('VIOLETA', 'SHE')
# Eliminar filas donde 'des_line' es 'home'
product_data_clean = product_data[product_data['des_line'] != 'HOME']


### Clean outfits data 

In [5]:
item_outfit_mod = outfit_data.groupby('cod_outfit')['cod_modelo_color'].agg(list).reset_index()

In [6]:
# Crear un diccionario para mapear 'cod_modelo_color' a 'des_product_type'
mapping_dict = dict(zip(product_data['cod_modelo_color'], product_data['des_product_aggregated_family']))

In [7]:
# Aplicar la función a cada fila en la columna 'cod_modelo_color'
def replace_codes( codes_list):
    return [mapping_dict.get(code, code) for code in codes_list]
item_outfit_mod['cod_modelo_color'] = item_outfit_mod['cod_modelo_color'].apply(replace_codes)


In [8]:
# Aplicar la función a cada fila y filtrar el DataFrame
cod_outfit_filtrado = item_outfit_mod[~item_outfit_mod['cod_modelo_color'].apply(contiene_numeros)]

In [9]:
elementos_interes = ['Trousers & leggings', 'Jeans', 'Dresses and jumpsuits', 'Shirts', 'Sweaters and Cardigans', 'Skirts and shorts', 'Tops', 'Jackets and Blazers', 'Coats and Parkas', 'T-shirts', 'Swim and intimate', 'Fragance', 'Decor', 'Bedroom', 'Dinning Room', 'Footwear']

In [10]:
# Aplicar la función a cada lista en 'cod_modelo_color' y filtrar el DataFrame
cod_outfit_mas_filtrado = cod_outfit_filtrado[~cod_outfit_filtrado['cod_modelo_color'].apply(lambda lista: tiene_mas_de_dos(lista, elementos_interes))]

In [11]:
# Aplicar la función de filtrado
cod_outfit_real = cod_outfit_mas_filtrado[cod_outfit_mas_filtrado['cod_modelo_color'].apply(cumple_condiciones)]

In [12]:
tabla_final = cod_outfit_real.drop(['cod_modelo_color'], axis=1)
tabla_final = tabla_final.reset_index(drop=True)

In [13]:
tabla_final = pd.merge(tabla_final, outfit_data, on='cod_outfit', how='left')
tabla_final_outfits = tabla_final

In [14]:
tabla_final = pd.merge(tabla_final_outfits, product_data_clean, on='cod_modelo_color', how='left')

In [15]:
# Obtener la lista de todas las columnas
columnas = tabla_final.columns.tolist()

# Índices de las columnas a intercambiar
idx1 = columnas.index('cod_outfit')
idx2 = columnas.index('cod_modelo_color')

# Intercambiar los nombres de las columnas
columnas[idx1], columnas[idx2] = columnas[idx2], columnas[idx1]

# Reordenar el DataFrame
tabla_final = tabla_final[columnas]

In [16]:
tabla_final

,cod_modelo_color,cod_outfit,cod_color_code,des_color_specification_esp,des_agrup_color_eng,des_sex,des_age,des_line,des_fabric,des_product_category,des_product_aggregated_family,des_product_family,des_product_type,des_filename
0,51000622-02,1,02,OFFWHITE,WHITE,Female,Adult,SHE,K-CIRCULAR,Tops,Tops,Tops,Top,datathon/images/2019_51000622_02.jpg
1,43067759-01,1,01,BLANCO,WHITE,Female,Adult,SHE,P-PLANA,Bottoms,Skirts and shorts,Skirts,Skirt,datathon/images/2019_43067759_01.jpg
2,53060518-02,1,02,OFFWHITE,WHITE,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Footwear,Footwear,Sandals,datathon/images/2019_53060518_02.jpg
3,53030594-08,1,08,BEIGE,WHITE,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Accessories,Bags,Handbag,datathon/images/2019_53030594_08.jpg
4,43077762-01,1,01,BLANCO,WHITE,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Accessories,Jewellery,Necklace,datathon/images/2019_43077762_01.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30556,53080809-OR,7842,OR,ORO,WHITE,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Accessories,Jewellery,Earrings,datathon/images/2019_53080809_OR.jpg
30557,53080716-OR,7842,OR,ORO,WHITE,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Accessories,Jewellery,Bracelet,datathon/images/2019_53080716_OR.jpg
30558,53023750-99,7842,99,NEGRO,GREY,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Accessories,Bags,Handbag,datathon/images/2019_53023750_99.jpg
30559,57035922-99,7842,99,NEGRO,GREY,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Footwear,Footwear,Ankle Boots,datathon/images/2019_57035922_99.jpg


In [17]:
tabla_final_outfits

,cod_outfit,cod_modelo_color
0,1,51000622-02
1,1,43067759-01
2,1,53060518-02
3,1,53030594-08
4,1,43077762-01
...,...,...
30556,7842,53080809-OR
30557,7842,53080716-OR
30558,7842,53023750-99
30559,7842,57035922-99


In [18]:
df_out_product = tabla_final_outfits.groupby(by='cod_outfit')['cod_modelo_color'].agg(list).reset_index()
df_out_product

,cod_outfit,cod_modelo_color
0,1,"[51000622-02, 43067759-01, 53060518-02, 530305..."
1,2,"[53060518-02, 51000622-06, 41041002-02, 530605..."
2,3,"[43075794-OR, 43005802-OR, 51000622-70, 410788..."
3,4,"[43075794-OR, 43005802-OR, 43073737-OR, 430637..."
4,5,"[53060518-02, 43005802-OR, 43073737-OR, 430257..."
...,...,...
5692,7836,"[53080809-OR, 53015777-99, 53087684-32, 579659..."
5693,7837,"[57085919-99, 53021099-OR, 53063786-99, 579659..."
5694,7840,"[53080809-OR, 51080862-99, 53053793-OR, 570067..."
5695,7841,"[53080809-OR, 53080716-OR, 53023750-99, 570359..."


## Generate new outfits

In [19]:
grouped_prendas = tabla_final.groupby(['des_color_specification_esp', 'des_product_type'])

# Crear un diccionario para mapear cada grupo de similitud a las prendas correspondientes
similarity_groups = {}
for name, group in grouped_prendas:
    similarity_groups[name] = set(group['cod_modelo_color'].to_list())


# Mostrar algunos grupos de similitud como ejemplo
example_groups = {k: similarity_groups[k] for k in tuple(similarity_groups)[:5]}
example_groups

{('AGUA', 'Blazer'): {'41095814-45'},
 ('AGUA', 'Crossbody bag'): {'57062510-45'},
 ('AGUA', 'Dress'): {'53040590-45',
  '53040892-45',
  '53080589-45',
  '57050307-45'},
 ('AGUA', 'Handbag'): {'53050624-45'},
 ('AGUA', 'Jacket'): {'51053748-45'}}

In [20]:
# Generar nuevas combinaciones de outfits para un subconjunto del dataset
new_outfits_subset = []
for outfit in df_out_product['cod_modelo_color'].iloc[:1]:  # GUMER CAMBIAR A 100
    new_outfits_subset.extend(generate_new_outfits_optimized(tabla_final,outfit, similarity_groups))

# Eliminar posibles duplicados
unique_new_outfits_subset = [list(x) for x in set(tuple(x) for x in new_outfits_subset)]

# Crear nuevos códigos de outfit para las nuevas combinaciones
new_cod_outfit_start_subset = df_out_product['cod_outfit'].max() + 1
new_cod_outfit_end_subset = new_cod_outfit_start_subset + len(unique_new_outfits_subset)
new_cod_outfits_subset = list(range(new_cod_outfit_start_subset, new_cod_outfit_end_subset))

# Crear el dataframe para las nuevas combinaciones
df_new_outfits_subset = pd.DataFrame({'cod_outfit': new_cod_outfits_subset, 'cod_modelo_color': unique_new_outfits_subset})

# Concatenar con el dataframe original, manteniendo el orden
df_out_product_expanded_subset = pd.concat([df_out_product, df_new_outfits_subset], ignore_index=True)

# Mostrar los últimos registros del dataframe expandido como verificación
df_out_product_expanded_subset.tail()


,cod_outfit,cod_modelo_color
5992,8138,"[51000622-02, 43067759-01, 53060518-02, 530305..."
5993,8139,"[51000622-02, 43067759-01, 53060518-02, 530305..."
5994,8140,"[51000622-02, 43067759-01, 53060518-02, 530305..."
5995,8141,"[51000622-02, 43067759-01, 53060518-02, 530305..."
5996,8142,"[53030876-02, 43067759-01, 53060518-02, 530305..."


In [21]:
out = []
col2 = []
for cod_out, list_prod in zip(df_out_product_expanded_subset['cod_outfit'],df_out_product_expanded_subset['cod_modelo_color']):
    for i in list_prod:
        out.append(cod_out)
        col2.append(i)

df_outfits = pd.DataFrame({'cod_outfit': out, 'cod_modelo_color': col2})


In [22]:
nuevos = df_outfits[df_outfits['cod_outfit']> 7482]

In [23]:
for t, n in zip(nuevos['cod_outfit'],nuevos['cod_modelo_color']):
    fila = tabla_final[tabla_final['cod_modelo_color'] == n].iloc[0,:]
    fila['cod_outfit'] =  t
    fila = pd.DataFrame([fila])
    tabla_final = pd.concat([tabla_final, fila], ignore_index=True)

## Save Data

In [24]:
tabla_final.to_csv("tabla_final.csv")
df_out_product_expanded_subset.to_csv("outfits.csv")